In [ ]:
using Pkg
Pkg.build("HDF5")

In [ ]:
Pkg.add("ITensors")

In [ ]:
using PastaQ
using Sockets
using ITensors

In [ ]:
function PastaQ.gate(::GateName"R"; theta::Real, phi::Real)
    [
        cos(theta/2)    (-im * exp(-im * phi) * sin(theta/2))
        (-im * exp(im * phi) * sin(theta/2))     cos(theta/2)
    ]
end

In [ ]:
N=1;
one_qubit_layer = Tuple[]
for j in 1:N
    gate = ("R", j, (theta=2pi*rand(), phi=2pi*rand()))
    push!(one_qubit_layer, gate)
end
gate

In [ ]:
function PastaQ.gate(::GateName"M"; Theta::Real)
    [
        cos(Theta)    0    0    (-im * sin(Theta))
        0    cos(Theta)    (-im * sin(Theta))    0
        0    (-im * sin(Theta))    cos(Theta)    0
        (-im * sin(Theta))    0    0    cos(Theta)
    ]
end

In [ ]:
function run(N, depth, nshot)
    # Random circuit.
    gates = Vector{Tuple}[]

    for i in 1:depth
        one_qubit_layer = Tuple[]
        two_qubit_layer = Tuple[]

        for j in 1:N
            gate = ("R", j, (theta=2pi*rand(), phi=2pi*rand()))
            push!(one_qubit_layer, gate)
        end

        # Alternate start qubit for pairs.
        idx_first = i % 2 + 1

        for j in idx_first:2:(N-1)
            gate = ("M", (j, j+1), (Theta=2pi*rand(),))
            push!(two_qubit_layer, gate)
        end

        push!(gates, one_qubit_layer)
        push!(gates, two_qubit_layer)
    end

    #psi = runcircuit(N, gates)
    data, psi = getsamples(N, gates, nshots,local_basis = ["Z"])
    
end


In [30]:
2% 2 + 1

1

In [34]:
function run(N, depth)
    # Random circuit.
    gates = Vector{Tuple}[]

    for i in 1:depth
        one_qubit_layer = Tuple[]
        two_qubit_layer = Tuple[]

        for j in 1:N
            gate = ("R", j, (theta=2pi*rand(), phi=2pi*rand()))
            push!(one_qubit_layer, gate)
        end

        # Alternate start qubit for pairs.
        idx_first = i % 2 + 1

        for j in idx_first:2:(N-1)
            gate = ("M", (j, j+1), (Theta=2pi*rand(),))
            push!(two_qubit_layer, gate)
        end

        push!(gates, one_qubit_layer)
        push!(gates, two_qubit_layer)
    end

    #psi = runcircuit(N, gates)
    gates
    
end
run(2, 2)

4-element Vector{Vector{Tuple}}:
 [("R", 1, (theta = 5.911718972950587, phi = 1.5174000035352968)), ("R", 2, (theta = 5.473937151929069, phi = 1.254731953436359))]
 []
 [("R", 1, (theta = 5.979008133035394, phi = 4.849191695115465)), ("R", 2, (theta = 2.9605792237329314, phi = 2.6423107604779537))]
 [("M", (1, 2), (Theta = 3.1035868756704277,))]

In [31]:
1 % 2 + 1

2

In [ ]:
nshots=100
N=20;
depth=5;
data,ρ= run(N, depth,nshots);

In [ ]:
function proj(j,size)
    a=[data[i][2] for i=(j-1)*size+1:(j)*size]
    a1=count(i->(i==1),a)/size
    a1
end
Mdata=[proj(i,nshots) for i=1:20] 

In [ ]:
Pkg.add("Plots")
using Plots
plot(Mdata) 